In [84]:
import pandas as pd

In [85]:
truncated_data = pd.read_csv("../data/truncated_dataset.csv").drop(["Unnamed: 0", "index"], axis=1)

truncated_data['step'] = truncated_data["step"].astype(int)
truncated_data['anglez'] = truncated_data["anglez"].astype(float)
truncated_data['enmo'] = truncated_data["enmo"].astype(float)
truncated_data['awake'] = truncated_data["awake"].astype(bool)

truncated_data['timestamp'] = pd.to_datetime(truncated_data['timestamp'], utc=True)

truncated_data["timestamp"].nunique() == truncated_data.shape[0]
truncated_data.set_index("timestamp", inplace=True)

In [86]:
truncated_data.head()

,series_id,step,anglez,enmo,awake
timestamp,,,,,
2019-02-08 17:00:00+00:00,349c5562ee2c,0,-88.683701,0.0,True
2019-02-08 17:00:05+00:00,349c5562ee2c,1,-88.683701,0.0,True
2019-02-08 17:00:10+00:00,349c5562ee2c,2,-88.683701,0.0,True
2019-02-08 17:00:15+00:00,349c5562ee2c,3,-88.683701,0.0,True
2019-02-08 17:00:20+00:00,349c5562ee2c,4,-88.683701,0.0,True


In [87]:
truncated_data.groupby("series_id")["step"].count()

series_id
349c5562ee2c     37080
a9e5f5314bcb    155160
c535634d7dcd    136080
Name: step, dtype: int64

In [88]:
RESAMPLE_MINUTES = "10min"

In [89]:
columns_for_resample = truncated_data[["anglez", "enmo", "series_id"]]
series = truncated_data["series_id"].unique()

res_dataframes = []
for s in series:
    res_dataframes.append(
        columns_for_resample[columns_for_resample["series_id"] == s][["anglez", "enmo"]].resample(RESAMPLE_MINUTES).mean()
        )
res = pd.concat(res_dataframes, axis=0)

In [90]:
target_columns = truncated_data.drop(["anglez", "enmo"], axis=1)
res = pd.merge(res, target_columns, how="inner", on="timestamp")
res.head()

,anglez,enmo,series_id,step,awake
timestamp,,,,,
2019-02-08 17:00:00+00:00,-88.683701,0.000000,349c5562ee2c,0,True
2019-02-08 17:10:00+00:00,-88.683701,0.000000,349c5562ee2c,120,True
2019-02-08 17:20:00+00:00,-81.866323,0.013463,349c5562ee2c,240,True
2019-02-08 17:30:00+00:00,-0.461967,0.122138,349c5562ee2c,360,True
2019-02-08 17:40:00+00:00,28.880279,0.030445,349c5562ee2c,480,True


In [91]:
res.to_csv("../data/resampled_dataset.csv")